# Introduccion  

El presente documento contiene el informe con el trabajo realizado para la primra práctica de Computación Evolutiva por el alumno Andrés Mañas Mañas.  

El lenguaje de programación elegido para la realización de la práctica es **Clojure** (un dialecto de Lisp). He elegido este lenguaje porque hace muy sencillo la representación de conocimiento como estructuras de datos muy fácilmente manipulables. Es además mi lenguaje favorito.   

En el material entregado para esta práctica se incluye el archivo **README.pdf**. Es del todo recomendable leer tal archivo en primer lugar para:  
    - documentarse sobre cómo disponer de un entorno en el que ejecutar el código fuente de la práctica
    - visualizar rápidamente la estructura de árbol con todos los documentos que se incluyen en la práctica  
  
Parte de los entrgables de la práctica consisten en un sistema que permita ver la evolución del algoritmo y los resultados alcanzdos. No son tantas las librerías que permiten visualizar gráficos en Clojure. Si acaso la más destacada sea [http://incanter.org/](http://incanter.org/). Sin embargo es una librería pesadísima, y he preferido no agregarla como dependencia del proyecto. Además de lo desagradable que es que al ejecutar los algoritmos se vayan abriendo popups en background.  

Por eso me he tomado la libertad de desplegar en un pequeño servidor que tengo en AWS un web service en php y un panel html que permite ver en tiempo real la evolución de una ejecución del algoritmo de aprendizaje.  

Cuando se lanza una ejecución de un experimento, de forma transparente se envían periódicamente estadísticas del avance del algoritmo a [http://amanas.ml/ce/resources/backgroun.php](http://amanas.ml/ce/resources/backgroun.php)  

Por otro lado, si simultáneamente tenemos abierta en un navegador la página [http://amanas.ml/ce/resources/dashboard.html](http://amanas.ml/ce/resources/dashboard.html), podrémos ir viendo en tiempo real y de forma gráfica la evolución del aprendizaje del experimento que estemos ejecutando en cada momento.  

## MUY IMPORTANTE !!!  
Hay que disponer de conexión a internet y abrír [http://amanas.ml/ce/resources/dashboard.html](http://amanas.ml/ce/resources/dashboard.html) cuando ustedes vayan a probar los algoritmos que les entrego con esto materiales.

Cada vez que se inicia un experimento, los datos del estado se resetean. Y no vuelven a resetearse hasta que no se inicia un nuevo experimento. Por eso si ustedes van a [http://amanas.ml/ce/resources/dashboard.html](http://amanas.ml/ce/resources/dashboard.html) y ven que los gráficos contienen datos, símplemente hay que interpretarlos como el estado en el que quedó la última ejecución del algoritmo de aprendizaje.


# Descripción del problema y requisitos  

En la primera práctica se nos pide que implementedmos desde cero un algoritmo genético para la resolución del **Problema de la Mochila Binario**.  

El **Problema de la Mochila Binario** se describe así: Dada una mochila con cierta “capacidad” y varios objetos con cierto “volumen” y “valor”, se trata de determinar qué objetos hay que introducir en la mochila para maximizar el valor total de los objetos contenidos en la misma. Obviamente, se debe cumplir que la suma de los volúmenes de los objetos introducidos en la mochila no exceda la capacidad de ésta.  

Los requisito pedidos son:
    - representación binaria de los individuos
    - función fitness basada en ordenamiento por ratio de los objetos y de forma que trate de forma adecuada aquellos individuos que representan soluciones no factibles al problema de la mochila binario
    - población inicial aleatoria
    - selección de padres por torneo
    - cruce por punto con probabilidad dada
    - mutación de todos los genes con una probabilidad dada
    - selección de supervivientes atendiendo a un modelo generaciónal con elitismo
    
La implementación del algoritmo que satisface las especificacione indicadas anteriormente pueden ustedes encontrarlas en el archivo src/ce/p1.clj  

En la implementación que entrego todos los metaparámetros del algoritmo son configurables al lanzarse. De este modo, la función que lleve a cabo la ejecución de un experimento es:

```clojure
(defn go-live [config]
   ...
```    

a la que se le pasa un mapa con los parámetros de configuración. Estos parámetros son los siguientes:  

```clojure
 {;; El tamaño de la mochila
  :pack-size 755
  ;; Los objetos - los inicializaré después de definir la función arrange-objects
  :objects objects
  ;; Tamaño inicial de la población
  :population-size 50
  ;; Número de individuos seleccionados por ronda en selección por torneo
  :tournament-round-size 5
  ;; Torneo con o sin reemplazamiento
  :replacement true
  ;; Probabilidad de activación de los genes cuando se genera un individuo
  :rand-gen-prob 5/10
  ;; Probabilidad utilizada para selección estocástica
  :first-stochastic-prob 8/10
  ;; Pobabilidad de mezcla en crossover
  :crossover-prob 5/10
  ;; Número de generaciones tope para el experimento
  :generations-threshold 200
  ;; Umbral de fitness relativo que, una vez alcanzado, da por finalizado el experimento
  ;; Se calcula como el fitness del mejor individuo de la generación / tamaño de la mochila
  :fitness-threshold 1
  ;; Número de generaciones sin mejora del fitness que se permiten antes de dar por acabado el experimento
  :blockage-delta 20
  ;; Cada cuantas generaciones se reporta el estado al web-service en la nube para su visualización
  :report-delta 1}
```  

Por ejemplo, una ejecución del algoritmo de aprendizaje podría hacerse así:  


```clojure
;; Genera un objeto interpretable por mi algoritmo a partir de sus propiedades.
;; - nam: nombre del objeto
;; - val: valor del objeto
;; - vol: volumen del objeto
(defn new-object
  ([[nam val vol]] (new-object nam val vol))
  ([nam val vol] {:nam nam :val val :vol vol}))

;; Genera un objeto aleatoriamente con valor entre 0 y max-val y
;; volumen entre 1 y max-vol.
;; - i: número de objeto
;; - max-val: valor máximo
;; - max-vol: volument máximo
(defn rand-object [i max-val max-vol]
  (new-object (str "object " i) (rand-int max-val) (inc (rand-int (dec max-vol)))))

;; Plantilla para experimentos con datos autogenerados
(go-live {:pack-size 755
          :objects (map #(rand-object % 20 20) (range 200))
          :population-size 50
          :tournament-round-size 5
          :replacement true
          :rand-gen-prob 5/10
          :first-stochastic-prob 8/10
          :crossover-prob 5/10
          :generations-threshold 200
          :fitness-threshold 1
          :blockage-delta 20
          :report-delta 1})
```    

Y con estas aclaraciones, creo que ya podemos pasar a comentar las experimentaciones que he realizado siguiendo el guión de la práctica.
